In [7]:
# References
# https://www.kaggle.com/code/vbmokin/nh4-tsaug-time-series-augmentation/notebook

In [53]:
from datetime import datetime, timedelta

import tsaug
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse
from tsaug.visualization import plot

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

In [54]:
date_format = '%Y-%m-%d'
def date_parser(x):
	return datetime.strptime(x, date_format)

# map_date = np.vectorize(lambda x: datetime.fromtimestamp(x/1000.0))

In [55]:
file = './data/gcp_cost.csv'
# Date column index is 2
df = pd.read_csv(file, sep=',', header=0, parse_dates=[2],
                  date_parser=date_parser,  dtype={'gcpId': object})
df = df[['date', 'costInUsd', 'gcpId']].sort_values('date')
df = df.dropna()
instanceId = '7994828870141789187'
df_i = df[df['gcpId'] == instanceId]
df_i

date  costInUsd                gcpId
805  2022-05-11   0.272394  7994828870141789187
3295 2022-05-12   0.920838  7994828870141789187
4041 2022-05-13   0.828322  7994828870141789187
2256 2022-05-14   0.875552  7994828870141789187
2724 2022-05-15   0.869427  7994828870141789187
...         ...        ...                  ...
1229 2022-09-23   0.881169  7994828870141789187
1602 2022-09-24   0.927295  7994828870141789187
418  2022-09-25   0.829429  7994828870141789187
3223 2022-09-26   0.401194  7994828870141789187
727  2022-09-27   0.000212  7994828870141789187

[140 rows x 3 columns]

In [56]:
start_date = df_i.min(axis=0)['date']
end_date = df_i.max(axis=0)['date']
days_range = end_date - start_date + timedelta(days=1)
print(f"{start_date} -> {end_date}: {days_range} days", )

2022-05-11 00:00:00 -> 2022-09-27 00:00:00: 140 days 00:00:00 days


In [57]:
X = np.arange(len(df_i))
X_date = df_i['date'].map(lambda x: int(x.timestamp()*1000)).tolist()

In [58]:
Y = df_i['costInUsd'].tolist()
Y = np.array(Y)

In [59]:
### AUGMENTATION HERE
def add_noise(Y, X, scale=0.1):
    Y_aug_noise, X_aug_noise = tsaug.AddNoise(scale=scale).augment(Y, X)
    return Y_aug_noise, X_aug_noise

def add_drift(Y, X, max_drift=0.5, n_drift_points=5):
    Y_aug_drift, X_aug_drift = tsaug.Drift(max_drift=max_drift, n_drift_points=n_drift_points).augment(Y, X)
    return Y_aug_drift, X_aug_drift

In [60]:
len(Y)

140

In [61]:
Y_aug_noise, X_aug_noise = add_noise(Y, X)
Y_aug_drift, X_aug_drift = add_drift(Y, X)

In [62]:
def draw_plot_px2(title_plot, X_aug, Y_aug, label1, X_aug2, Y_aug2, label2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_aug, y=Y_aug,
                        mode='lines',
                        name=label1))
    fig.add_trace(go.Scatter(x=X_aug2, y=Y_aug2,
                        mode='lines',
                        name=label2))
    fig.add_trace(go.Scatter(x=X, y=Y,
                        mode='lines',
                        name='Origin data'))
    fig.update_layout(title=title_plot)
    fig.show()

In [63]:
draw_plot_px2('GCP_cost_March-Sept',X_aug_noise, Y_aug_noise, 'after_add_noise', X_aug_drift, Y_aug_drift, 'after_add_drift')

In [64]:
NUM_OF_PERIODS = 4
X_date_extended1 = pd.date_range(start_date -(days_range) * NUM_OF_PERIODS,start_date,freq='d').map(lambda x: int(x.timestamp()*1000))
X_date_extended = np.hstack([X_date_extended1, X_date])[1:]

In [66]:
# Start executing add extended_Y, add noise to each result
extended_Y = []
cur_Y = Y
for i in range(0, NUM_OF_PERIODS):
    cur_Y, X = add_noise(cur_Y, X)
    extended_Y.append(cur_Y)

Y_extended = np.hstack([Y] + extended_Y)

In [67]:
vfunc = np.vectorize(lambda x: datetime.fromtimestamp(x/1000.0))
plt.plot(vfunc(X_date_extended), Y_extended)

In [75]:
new_df = pd.DataFrame({'X': list(map(lambda x: datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d'),X_date_extended))})
new_df['Y'] = pd.Series(Y_extended) 

In [76]:
new_df

X         Y
0    2020-10-29  0.272394
1    2020-10-30  0.920838
2    2020-10-31  0.828322
3    2020-11-01  0.875552
4    2020-11-02  0.869427
..          ...       ...
695  2022-09-23  0.918605
696  2022-09-24  0.703406
697  2022-09-25  0.302408
698  2022-09-26  1.420249
699  2022-09-27  0.037547

[700 rows x 2 columns]

In [77]:
new_df.to_csv('./data/final_augemented_gcp_cost.csv')